In [18]:
# from google.colab import drive
# drive.mount('/content/drive')

# !tar -xf /content/drive/My\ Drive/conda_colab.tar.gz -C ../

# import os
# path = '/content/miniconda3/bin:' + os.environ['PATH']
# %env PATH=$path
# %env PYTHONPATH=
# import sys
# _ = sys.path.append("/content/miniconda3/lib/python3.7/site-packages")

In [19]:
# This is supposed to make sure if you edit .py files
# colab will pick up on it
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
!if [[ ! -d PROJECT ]]; then git clone https://github.com/iggy2k/CSC490-Project-Winter-2025.git PROJECT; fi

In [ ]:
import sys
sys.path.insert(1, 'PROJECT/PIGEON')
print(sys.path)

['/Users/iggy/miniforge3/envs/3-8-10/lib/python38.zip', 'PROJECT/PIGEON', '/Users/iggy/miniforge3/envs/3-8-10/lib/python3.8', '/Users/iggy/miniforge3/envs/3-8-10/lib/python3.8/lib-dynload', '', '/Users/iggy/miniforge3/envs/3-8-10/lib/python3.8/site-packages', '/Users/iggy/miniforge3/envs/3-8-10/lib/python3.8/site-packages/setuptools/_vendor']


In [ ]:
!pip install -q condacolab

In [ ]:
import os
try:
    import condacolab
    condacolab.install() # expect a kernel restart
    IN_COLAB = True
    os.environ["IN_COLAB"] = "1"
    print('in colab')
except:
    IN_COLAB = False
    print('not in colab')
    os.unsetenv('IN_COLAB')

not in colab


In [ ]:
# Only use in colab
!if [ -v IN_COLAB ]; then yes | mamba create -q --name 3-8-10 python=3.8.10; fi
!if [ -v IN_COLAB ]; then eval "$(conda shell.bash hook)"; conda activate 3-8-10; python --version; fi
!if [ -v IN_COLAB ]; then mamba env update -n 3-8-10 -f "./PROJECT/environment.yaml"; fi

In [ ]:
!pip install -q pycountry

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !tar -zcf conda_colab.tar.gz /content/miniconda3
# !cp conda_colab.tar.gz /content/drive/My\ Drive/

In [ ]:
from huggingface_hub import snapshot_download
import os

if not os.path.isdir('datasets/osv5m/train'):
    snapshot_download(repo_id="osv5m/osv5m", local_dir="datasets/osv5m", allow_patterns=[
        'images/train/00.zip',
        'images/test/00.zip',
        '*.csv'
        ], repo_type='dataset')

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

train.csv:   3%|2         | 83.9M/2.92G [00:00<?, ?B/s]

00.zip:   4%|3         | 83.9M/2.25G [00:00<?, ?B/s]

test.csv:  63%|######3   | 73.4M/116M [00:00<?, ?B/s]

00.zip:   3%|2         | 73.4M/2.52G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
!pip install -q global-land-mask

In [ ]:
import csv
import numpy as np
import pandas as pd
from shapely.geometry import Point

import geopandas as gpd
from geopandas import GeoDataFrame

import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import zipfile
from time import sleep

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from torchvision import datasets, transforms

from PIL import Image
import pandas as pd

from tqdm.notebook import trange, tqdm

from global_land_mask import globe

/Users/iggy/miniforge3/envs/3-8-10/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [ ]:
for root, dirs, files in os.walk("datasets/osv5m"):
    for file in files:
        if file.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(root, file), 'r') as zip_ref:
                for member in tqdm(zip_ref.infolist(), desc=f'Extracting {os.path.join(root, file)}'):
                  zip_ref.extract(member, root)
            os.remove(os.path.join(root, file))

In [ ]:
import pycountry

# Debug
MAX_ITEMS = 1000

class ImageCoordinateDataset(Dataset):
    def __init__(self, csv_file, image_dirs, transform=None):
        self.data = []
        self.files = []
        self.images = []
        self.skipped = 0
        self.csv = csv_file
        self.transform = transform

        for image_dir in image_dirs:
          print('Reading', image_dir)

          self.files.extend([f"{image_dir}/{f}" for f in listdir(image_dir) if isfile(join(image_dir, f))])

          print(f'Found {len(self.files)} files.')

          if not os.path.isdir('datasets/osv5m/'):
            os.makedirs('datasets/osv5m/')
          if isfile(f"{csv_file}_filtered.csv"):
            self.df = pd.read_csv(f"{csv_file}_filtered.csv", index_col=False)
          else:
            self.df = pd.concat([chunk for chunk in tqdm(pd.read_csv(self.csv, chunksize=5000, usecols=['id', 'latitude', 'longitude', 'country'], index_col=False), desc='Loading data')])

          print(f'Found {len(self.df)} csv entries.')

          self.df['country'] = self.df['country'].apply(lambda x: pycountry.countries.get(alpha_2=x).name if pycountry.countries.get(alpha_2=x) else x)

          new = pd.DataFrame(columns=['id', 'latitude', 'longitude', 'country'])
          i = 0
          for full_path in tqdm(self.files, total=len(self.files), desc='Processing files'):
              image_name = str(Path(full_path).stem)

              try:
                row = self.df[self.df['id'] == int(image_name)].iloc[0]
              except:
                pass
              new.loc[i] = row
              lat = row['latitude']
              lon = row['longitude']

              # Remove mislaballed images (ocean pictures?)
              # if not globe.is_land(float(lat), float(lon)):
              #   self.skipped += 1
              #   continue
              self.data.append(np.array([str(full_path), float(lat), float(lon)]))
              i += 1

          self.df = new
        if MAX_ITEMS is not None:
          print(f'Keeping {MAX_ITEMS} items')
          self.df = self.df[:MAX_ITEMS]
          self.data = self.data[:MAX_ITEMS]
          self.files = self.files[:MAX_ITEMS]

        self.df.to_csv(f"{csv_file}_filtered.csv", index=False)

        print(f'Dataset ready, {len(self.files)} files.')
        print(f'Skipped {self.skipped} non-land files.')


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx][0]
        coordinates = (float(self.data[idx][1]), float(self.data[idx][2]))
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(coordinates, dtype=torch.float32)


In [ ]:
# https://www.geeksforgeeks.org/how-to-normalize-images-in-pytorch/
# The mean and std of ImageNet are: mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].
mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)
normalize = transforms.Normalize(mean.tolist(), std.tolist())

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

In [ ]:
BATCH_SIZE = 32

In [ ]:
train_dataset = ImageCoordinateDataset(csv_file='datasets/osv5m/train.csv', image_dirs=[
                                      'datasets/osv5m/images/train/00',
                                      ],\
                                 transform=transform)
val_dataset = ImageCoordinateDataset(csv_file='datasets/osv5m/test.csv', image_dirs=[
                                      'datasets/osv5m/images/test/00',
                                      ],\
                                 transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

Reading datasets/osv5m/images/train/00


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/osv5m/images/train/00'

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('using cuda')
else:
  device = torch.device("cpu")
  print('using cpu')

using cpu


In [ ]:
CLIP_MODEL = 'openai/clip-vit-large-patch14-336'

In [ ]:
from transformers import AutoModelForImageClassification, CLIPVisionModel
embed_model = CLIPVisionModel.from_pretrained(CLIP_MODEL)

In [ ]:
!mkdir -p data/geocells/
# Political boundaries (admin0 = country)
![ ! -f data/geocells/admin_2.geojson ] && wget --show-progress -O data/geocells/admin_2.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.geojson
![ ! -f data/geocells/admin_1.geojson ] && wget --show-progress -O data/geocells/admin_1.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.geojson
![ ! -f data/geocells/countries.geojson ] && wget --show-progress -O data/geocells/countries.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.geojson

In [ ]:
GEOCELL_PATH = 'data/geocells_yfcc.csv'

In [ ]:
# TODO: pre-generate all this, add to the repo and add option to load instead
import sys
sys.path.insert(0,'PROJECT/PIGEON/dataset_creation/geocell')
sys.path.insert(0,'PROJECT/PIGEON/')
from geocell_creation import *

df = train_dataset.df
geocells_file = Path(GEOCELL_PATH)
if not geocells_file.is_file():
    geocell_creator = GeocellCreator(df, GEOCELL_PATH)
    geocells = geocell_creator.generate()



In [ ]:
import torchvision
from torchvision import datasets, transforms

import torch
import pandas as pd
from torch import nn, Tensor
from torch.nn.parameter import Parameter
from collections import namedtuple
from preprocessing import haversine_matrix, smooth_labels
from models.layers import PositionalEncoder
from models.utils import ModelOutput
from config import *

#TODO: move over more stuff from PIGEON

class GeoLocationModel(nn.Module):
    def __init__(self):
        super(GeoLocationModel, self).__init__()

        # Save variables
        self.base_model = embed_model

        resnet = torchvision.models.resnet50(pretrained=True).to(device)
        for i, param in enumerate(resnet.parameters()):
          if i < 4:
            param.requires_grad = False


        # Setup
        self._set_hidden_size()
        geocell_path = GEOCELL_PATH
        self.lla_geocells = self.load_geocells(geocell_path)
        self.num_cells = self.lla_geocells.size(0)

        self.resnet = resnet
        num_feats = self.resnet.fc.in_features
        self.fc1 = nn.Linear(num_feats, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 2)



    def forward(self, x):
        features = self.resnet(x)

        x = self.fc1(features)
        x = self.relu(x)
        x = self.fc2(x)

        return x

    def load_geocells(self, path: str) -> Tensor:
        """Loads geocell centroids and converts them to ECEF format

        Args:
            path (str, optional): path to geocells. Defaults to GEOCELL_PATH.

        Returns:
            Tensor: ECEF geocell centroids
        """
        geo_df = pd.read_csv(path)
        lla_coords = torch.tensor(geo_df[['longitude', 'lat']].values)
        lla_geocells = nn.parameter.Parameter(data=lla_coords, requires_grad=False)
        return lla_geocells

    def _set_hidden_size(self):
        """
        Determines the hidden size of the model
        """
        if self.base_model is not None:
            try:
                self.hidden_size = self.base_model.config.hidden_size
                self.mode = 'transformer'

            except AttributeError:
                self.hidden_size = self.base_model.config.hidden_sizes[-1]
                self.mode = 'convnext'

model = GeoLocationModel()

# model = model.to(device)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/iggy/miniforge3/envs/3-8-10/lib/python3.8/site-packages/transformers/utils/import_utils.py", line 1184, in _get_module
  File "/Users/iggy/miniforge3/envs/3-8-10/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/iggy/miniforge3/envs/3-8-10/lib/python3.8/site-packages/transformers/models/auto/feature_extraction_auto.py", line 25, in <module>
    from ...feature_extraction_utils import FeatureExtractionMixin
  File "/Users

In [ ]:
from transformers import CLIPProcessor, CLIPModel

In [ ]:
clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-L-14-laion2B-s32B-b82K")
clip_processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-L-14-laion2B-s32B-b82K")
clip_model.to(device)

In [ ]:
# https://github.com/gastruc/osv5m/blob/main/models/losses.py
class HaversineLoss(nn.Module):
    def __init__(self):
        super(HaversineLoss, self).__init__()

    def forward(self, x, y):
        """
        Args:
            x: torch.Tensor Bx2
            y: torch.Tensor Bx2
        Returns:
            torch.Tensor: Haversine loss between x and y: torch.Tensor([B])
        Note:
            Haversine distance doesn't contain the 2 * 6371 constant.
        """
        lhs = torch.sin((x[:, 0] - y[:, 0]) / 2) ** 2
        rhs = (
            torch.cos(x[:, 0])
            * torch.cos(y[:, 0])
            * torch.sin((x[:, 1] - y[:, 1]) / 2) ** 2
        )
        a = lhs + rhs
        return torch.arctan2(torch.sqrt(a), torch.sqrt(1 - a))

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR

haversineLoss = HaversineLoss()
mseLoss = nn.MSELoss()
ceLoss = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = ExponentialLR(optimizer, gamma=0.9, verbose=True)

In [ ]:
!pip install -q livelossplot

In [ ]:
from livelossplot import PlotLosses

In [ ]:
from ipywidgets import Output
OUTPUT_CONTEXT = Output()
display(OUTPUT_CONTEXT)

In [ ]:
num_epochs = 5
avg_epoch_loss = []

plotlosses = PlotLosses(figsize=(10, 5))


for epoch in tqdm(range(num_epochs), total=num_epochs, desc='Epochs'):
  train_loss = []
  avg_loss = 0

  for images, coordinates in (pbar:= tqdm(train_dataloader, total=len(train_dataloader), desc=f'Epoch {epoch + 1} progress')):
    images, coordinates = images.cuda(), coordinates.cuda()

    optimizer.zero_grad()
    outputs = model(images)

    loss_haversine = haversineLoss(outputs, coordinates).mean(dim=-1)

    mse = mseLoss(outputs, coordinates)
    cross_entropy = ceLoss(outputs, coordinates)

    pbar.set_postfix_str(f'Loss: {loss_haversine:.5f}; \
                               \nOut: {outputs.mean(dim=0).tolist()}\
                               \nExpected: {coordinates.mean(dim=0).tolist()}')
    train_loss.append(loss_haversine.detach())

    with OUTPUT_CONTEXT:
      plotlosses.update({
          'Haversine': loss_haversine.item(),
          'MSE': mse.item(),
          'Cross Entropy': cross_entropy.item()
      })
      plotlosses.send()

    loss_haversine.backward()
    optimizer.step()

  scheduler.step()
  avg_loss = sum(train_loss) / len(train_dataloader)
  avg_epoch_loss.append(avg_loss)
  print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

In [ ]:
import matplotlib.pyplot as plt
avg_epoch_loss = torch.tensor(avg_epoch_loss, device =  'cpu')
plt.figure(figsize=(7, 5))
plt.plot(range(1, len(avg_epoch_loss) + 1), avg_epoch_loss, label='Loss', color='blue', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epoch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
model.eval()

In [ ]:
avg_val_haversine_loss = 0.0
batch_haversine = []

with torch.no_grad():  # Disable gradient calculation for validation
    for images, coordinates in tqdm(val_dataloader, total=len(val_dataloader), desc=f'Validating'):
        # Forward pass
        images, coordinates = images.cuda(), coordinates.cuda()
        outputs = model(images)

        loss_haversine = haversineLoss(outputs, coordinates).mean(dim=-1)
        batch_haversine.append(loss_haversine)
        avg_val_haversine_loss += loss_haversine

# Calculate average loss and haversine
avg_val_haversine_loss /= len(val_dataloader)

print(f'Validation Haversine Loss: {avg_val_haversine_loss:.4f}')

In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter

# Back from gpu
batch_haversine = torch.tensor(batch_haversine, device =  'cpu')

# Remove outliers
batch_haversine = median_filter(batch_haversine, size=100)

plt.figure(figsize=(5, 5))
plt.subplot(1, 1, 1)
plt.plot(range(1, len(batch_haversine) + 1), batch_haversine, label='Validation haversine', color='green', marker='o')
plt.xlabel('Batch Number')
plt.ylabel('haversine')
plt.title('Validation haversine loss per Batch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Step 5: Visualize some predictions (optional)
import matplotlib.pyplot as plt
import random

url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"

unnormalize = transforms.Normalize((-mean / std).tolist(), (1.0 / std).tolist())

def plot_predictions(model, dataloader, num_samples=5):
    model.eval()
    with torch.no_grad():
        for images, coordinates in dataloader:

            images = images.cuda()

            outputs = model(images)

            images = images.cpu()
            outputs = outputs.cpu()

            rand_index = random.sample(range(0, len(images) - 1), min(num_samples, len(images) - 1))

            for i in range(min(num_samples, len(images))):
                i = rand_index[i]

                pred_lat, pred_lon = outputs[i].cpu().numpy()
                true_lat, true_lon = coordinates[i].numpy()

                haver_err = haversineLoss(
                                      torch.tensor(np.array([[pred_lon, pred_lat]]), dtype=torch.float32).deg2rad(),
                                      torch.tensor(np.array([[true_lon, true_lat]]), dtype=torch.float32).deg2rad(),
                                      )
                # Display the image
                img = images[i]
                img = unnormalize(img).permute(1, 2, 0).numpy()
                img = np.clip(img, 0, 1)

                plt.imshow(img)
                plt.title(f'Pred: ({pred_lat:.4f}, {pred_lon:.4f})\nTrue: ({true_lat:.4f}, {true_lon:.4f})\n Haversine: {haver_err}')
                plt.axis('off')

                # World map for better understanding of how bad our prediction is
                geometry = [Point(pred_lon, pred_lat), Point(true_lon, true_lat)]
                geo_df = GeoDataFrame(geometry = geometry)
                world = gpd.read_file(url)
                geo_df.plot(ax=world.plot(color="lightgrey", figsize=(10, 6)), marker='x', c=['red', 'green'], markersize=50);

                plt.show()
            break

In [ ]:
plot_predictions(model, val_dataloader, num_samples=10)